In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime as dt
from toolz import thread_first, take
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns=100
pd.options.display.width = 200

OUTPUT_DIR = os.path.join(os.path.expanduser('~'), 'Documents/output_wv_large/csv/')
ignore_these = ['observations', 'immunizations', 'claims_transactions', 'organizations', 'payers', 'payer_transitions', 'imaging_studies', 'supplies', 'allergies', 'devices', 'providers']

def get_file(name):
    return OUTPUT_DIR + name + '.csv'

def printer(data, sample=True, n=5, name=None, pprint=False):
    if name:
        print(name, data.shape)
    else:
        print(data.shape)
    print('-'*10)
    
    d = data.sample(min(n, data.shape[0])) if sample else data.head(n)
    if pprint:
        print(d)
    return d

print('List of datasets')
print('-'*10)
print(os.listdir(OUTPUT_DIR))

print('\n')
print('-'*50, '\n')
data = {}
for f in os.listdir(OUTPUT_DIR):
    fname = get_file(f.split('.')[0])
    fkey = f.split('.')[0].split('/')[-1]
    if fkey in ignore_these:
        continue
    data[fkey] = pd.read_csv(fname, dtype=str)
    if data[fkey].empty:
        continue
    printer(data[fkey], name=fkey, pprint=True)
    print('\n')
    
locals().update(data)

In [ ]:
def prep_patients(patients):
    patients = patients.copy(deep=True)
    patients['PATIENT_CT'] = 1
    patients['ZIP'] = patients['ZIP'].apply(lambda x: x.split('-')[0] if isinstance(x, str) else 'NA')
    patients['AGE'] = (dt.now() - pd.to_datetime(patients['BIRTHDATE'])).dt.days / 365.25
    patients['SEX'] = np.random.choice(['M', 'F', 'I'], size=patients.shape[0], p=[0.48, 0.49, 0.03])
    cols = ['Id', 'PATIENT_CT', 'ZIP', 'COUNTY', 'AGE', 'SEX', 'LAT', 'LON']
    return patients[cols].copy(deep=True)

def add_beacon_activity(patients):
    patients = patients.copy(deep=True)
    patients['ACTIVE_BEACON_PAST_YEAR_CT'] = np.random.choice([0, 1], size=patients.shape[0], p=[0.88, 0.12])
    patients['ACTIVE_BEACON_PAST_YEAR_PCT'] = np.where(patients[['OPIOID_DX', 'OPIOID_DX_PRIOR']].max(axis=1) == 1, patients['ACTIVE_BEACON_PAST_YEAR_CT'], np.NaN)
    return patients

def add_opioid_dx(patients, conditions):
    conditions = conditions.copy(deep=True)
    patients = patients.copy(deep=True)
    conditions['YEAR'] = pd.to_datetime(conditions['START']).dt.year
    conditions['OPIOID_DX'] = (conditions['CODE'] == '55680006')*1 # actually drug overdose
    this_year = conditions.loc[conditions['YEAR'] == dt.now().year].groupby('PATIENT', as_index=False)['OPIOID_DX'].max()
    prior_years = conditions.loc[conditions['YEAR'] < dt.now().year].groupby('PATIENT', as_index=False)['OPIOID_DX'].max()
    patients = patients.merge(this_year, how='left', left_on='Id', right_on='PATIENT')\
                       .drop('PATIENT', axis=1)\
                       .merge(prior_years, how='left', left_on='Id', right_on='PATIENT', suffixes=['', '_PRIOR'])\
                       .drop('PATIENT', axis=1)
    for col in [x for x in patients if 'OPIOID_' in x]:
        patients[col] = patients[col].fillna(0).copy(deep=True)
    return patients

def add_sex_metrics(patients):
    sex = pd.get_dummies(patients['SEX']).rename(columns={k: 'SEX_'+k+'_PCT' for k in patients['SEX'].unique()})
    return pd.concat([patients, sex], axis=1)

def add_age_metrics(patients):
    patients['AGE_GP'] = pd.cut(patients['AGE'], bins=[0, 18, 30, 50, 65, 120], labels=['UND_18', '18_30', '31_50', '51_65', '65_OVER'], include_lowest=True)
    age = pd.get_dummies(patients['AGE_GP'])
    age = age.rename(columns={k: 'AGE_'+k+'_CT' for k in age.columns})
    return pd.concat([patients, age], axis=1)

def add_hospitalizations(patients, encounters):
    patients = patients.copy(deep=True)
    
    def event(d):
        Y = d.DESCRIPTION.upper().replace('()', '').replace(' ', '_')
        X = d.REASONDESCRIPTION.upper().replace(' ', '_').replace('NA', '')
        return Y + '_DUE_TO_' + X if X else Y

    encounters = encounters[['PATIENT', 'DESCRIPTION', 'REASONDESCRIPTION', 'ENCOUNTERCLASS', 'TOTAL_CLAIM_COST', 'Id', 'STOP']]\
    .merge(patients[['ZIP', 'COUNTY', 'Id']],
                     how='left',
                     left_on='PATIENT', right_on='Id', suffixes=['_', ''])\
    .assign(DATE = lambda x: pd.to_datetime(x['STOP']))\
    .drop(['Id', 'STOP'], axis=1)\
    .drop_duplicates()\
    .rename(columns={'Id_': 'ENCOUNTER_ID'})\
    .fillna('NA')\
    .assign(EVENT = lambda x: x.apply(event, axis=1))\
    .assign(TOTAL_CLAIM_COST = lambda x: pd.to_numeric(x['TOTAL_CLAIM_COST']))
    
    encounters['SUDS_ENCOUNTER'] = encounters['EVENT'].str.contains('DRUG') * 1
    encounters['SUDS_HOSPITALIZATION'] = (encounters['EVENT'].str.contains('DRUG') & encounters['ENCOUNTERCLASS'].isin(['inpatient'])) * 1
    encounters['SUDS_HOSPITALIZATION_COST_SUM'] = np.where(encounters['SUDS_HOSPITALIZATION'] == 1, encounters['TOTAL_CLAIM_COST'], np.nan)
    encounters['SUDS_HOSPITALIZATION_COST_AVG'] = np.where(encounters['SUDS_HOSPITALIZATION'] == 1, encounters['TOTAL_CLAIM_COST'], np.nan)
    return encounters

def add_opioid_dx_rx(county):
    opioid_rates = pd.read_html(
        'https://www.cdc.gov/drugoverdose/rxrate-maps/county2019.html'
    )[0].rename(columns={
        'County': 'COUNTY',
        'County FIPS Code': 'FIPS',
        'Opioid Dispensing Rate per 100': 'OPIOID_RX_RATE'
    })
    opioid_rates['OPIOID_DX_RATE'] = opioid_rates.groupby('State')['OPIOID_RX_RATE']\
    .transform(lambda x: (x - x.mean()) / x.std()) 
    opioid_rates['OPIOID_DX_RATE'] *= 0.10 # 10% of population of WV has suds
    opioid_rates['OPIOID_DX_RATE'] = opioid_rates['OPIOID_DX_RATE'].abs()

    opioid_rates = opioid_rates.query("State == 'WV'").drop(['State', 'FIPS'], axis=1).reset_index(drop=True)
    return county.merge(opioid_rates, how='inner', on='COUNTY')

thread_first(patients, 
             prep_patients, 
             (add_opioid_dx, conditions),
             add_beacon_activity,
             add_sex_metrics,
             add_age_metrics,
             printer)

In [ ]:
test = thread_first(patients, 
             prep_patients, 
             (add_hospitalizations, encounters))

printer(test)

In [ ]:
GEO = 'COUNTY'
part1 = patients\
.pipe(prep_patients)\
.pipe(add_opioid_dx, conditions=conditions)\
.pipe(add_beacon_activity)\
.pipe(add_sex_metrics)\
.pipe(add_age_metrics)\
.assign(OVERALL=1)\
.groupby([GEO], as_index=False)\
.agg({
    'PATIENT_CT': 'count',
    'ACTIVE_BEACON_PAST_YEAR_CT': 'sum',
    'ACTIVE_BEACON_PAST_YEAR_PCT': 'mean',
    'OPIOID_DX_PRIOR': 'mean',
    'SEX_F_PCT': 'mean',
    'SEX_I_PCT': 'mean',
    'SEX_M_PCT': 'mean',
    'AGE_UND_18_CT': 'mean',
    'AGE_18_30_CT': 'mean',
    'AGE_31_50_CT': 'mean', 
    'AGE_51_65_CT': 'mean',
    'AGE_65_OVER_CT': 'mean'
})\
.sort_values('PATIENT_CT', ascending=False)\
.pipe(add_opioid_dx_rx)

printer(part1, sample=False)

In [ ]:
prep_patients(patients)['AGE'].describe()

In [ ]:
GEO = 'COUNTY'
part2 = patients\
.pipe(prep_patients)\
.pipe(add_opioid_dx, conditions=conditions)\
.pipe(add_beacon_activity)\
.pipe(add_sex_metrics)\
.pipe(add_age_metrics)\
.assign(OVERALL=1)\
.groupby(['OVERALL'], as_index=False)\
.agg({
    'PATIENT_CT': 'count',
    'ACTIVE_BEACON_PAST_YEAR_CT': 'sum',
    'ACTIVE_BEACON_PAST_YEAR_PCT': 'mean',
    'OPIOID_DX_PRIOR': 'mean',
    'SEX_F_PCT': 'mean',
    'SEX_I_PCT': 'mean',
    'SEX_M_PCT': 'mean',
    'AGE_UND_18_CT': 'mean',
    'AGE_18_30_CT': 'mean',
    'AGE_31_50_CT': 'mean', 
    'AGE_51_65_CT': 'mean',
    'AGE_65_OVER_CT': 'mean'
})\
.sort_values('PATIENT_CT', ascending=False)
part2

In [ ]:
# prepped = add_hospitalizations(prep_patients(patients), encounters)
years = range(prepped['DATE'].dt.year.min(), prepped['DATE'].dt.year.max())
jitter = pd.DataFrame({
    'YEAR': years,
    'JITTER': np.random.normal(0, 0.05, len(years))
})
jitter['JITTER'] *= jitter['YEAR']
jitter.YEAR = jitter.YEAR.astype(str)

part4 = patients\
.pipe(prep_patients)\
.pipe(add_hospitalizations, encounters=encounters)\
.assign(YEAR = lambda x: x['DATE'].dt.strftime('%Y'))\
.assign(OVERALL=1)\
.groupby([GEO, 'YEAR'], as_index=False)\
.agg({
    'SUDS_ENCOUNTER': 'sum',
    'SUDS_HOSPITALIZATION': 'sum',
})\
.assign(SUDS_HOSPITALIZATION_COST_AVG = 5297.72)\
.pipe(add_opioid_dx_rx)\
.merge(jitter, how='left', on='YEAR')\
.assign(SUDS_HOSPITALIZATION_COST_AVG = 
        lambda x: x['SUDS_HOSPITALIZATION_COST_AVG'] * x['OPIOID_DX_RATE'] * (x['JITTER']+ 1))\
.drop(['JITTER'], axis=1)

printer(part4, sample=False)

In [ ]:
part1.to_csv('output/county_level.csv', index=False)
# part4.to_csv('output/part4.csv', index=False)

# Fake the additional data we need
---

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
cost = pd.read_csv(
    '/Users/lpanda/Downloads/per-capita-economic-cost-of-opioid-use-disorder-and-fatal-overdose-by-state-2017.csv'
)\
.rename(columns={
   'Total Costs': 'SUDS_TOTAL_COST',
   'Per Capita Total Costs': 'SUDS_TOTAL_COST_AVG',
    'Location': 'STATE'
})\
.assign(POP = lambda x: x['SUDS_TOTAL_COST'] / x['SUDS_TOTAL_COST_AVG'])\
.assign(SUDS_OVERDOSE_COST_AVG = lambda x: x['Total Fatal Overdose Costs'] / x['POP'])\
.assign(SUDS_TOTAL_COST = lambda x: x['SUDS_TOTAL_COST'] * 0.13)

part5 = cost.loc[cost['STATE'] == 'West Virginia', ['STATE', 'POP', 'SUDS_TOTAL_COST', 'SUDS_TOTAL_COST_AVG', 'SUDS_OVERDOSE_COST_AVG']].reset_index(drop=True)
part5 = part5.join(part2)
part5.to_csv('output/state_level.csv', index=False)
part5

In [ ]:
part6 = pd.DataFrame({
    'YEAR': take(3000, sorted([round(x) for x  in np.random.normal(loc=2022, scale=3, size=10000) if x <= 2021])),
    'CABELL_COUNTY': take(3000, sorted([x for x  in np.random.normal(loc=7497, scale=250, size=10000) if x <= 7497])),
    'WEBSTER_COUNTY': take(3000, sorted([x for x  in np.random.normal(loc=7300, scale=586, size=10000) if x <= 7497])),
    'HANCOCK_COUNTY': take(3000, sorted([x for x  in np.random.normal(loc=6800, scale=137, size=10000) if x <= 7497]))
}).groupby('YEAR').mean() * 179000 * 0.6

part6.reset_index().to_csv('output/county_time_level.csv', index=False)
part6.plot(figsize=(10,6))

In [ ]:
# top prescribing providers
